In this notebook I'll reconstruct the process and results of the DTCR algorithm presented in the "Learning Representations for Time Series Clustering".

In [1]:
# Imports
import torch
from torch.utils.data import DataLoader
from Utilities.DTCR import DTCRModel, DTCRConfig
from Utilities.UCRParser import read_dataset
from Utilities.DRNN import BidirectionalDRNN

In [2]:
train_ds, test_ds = read_dataset("Two_Patterns")
bs = 20
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=bs, shuffle=True)

config = DTCRConfig()
config.class_num = train_ds.number_of_labels
config.input_size = train_ds[0][0].shape[1]
config.num_steps = train_ds[0][0].shape[0]
config.batch_size = bs


Loading the Two_Patterns dataset...
The dataset Two_Patterns was loaded.


In [3]:
dtcr_model = DTCRModel(config)
dtcr_model

DTCRModel(
  (_encoder): BidirectionalDRNN(
    (_regular_drnn): DRNN(
      (cells): Sequential(
        (0): GRU(1, 100)
        (1): GRU(100, 50)
        (2): GRU(50, 50)
      )
    )
    (_backwards_drnn): DRNN(
      (cells): Sequential(
        (0): GRU(1, 100)
        (1): GRU(100, 50)
        (2): GRU(50, 50)
      )
    )
  )
  (_decoder): DTCRDecoder(
    (_rnn): GRU(400, 400, batch_first=True)
    (_linear): Linear(in_features=400, out_features=1, bias=True)
  )
  (_classifier): Sequential(
    (0): Linear(in_features=400, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2, bias=True)
    (3): Softmax(dim=1)
  )
)

In [4]:
for epoch in range(5):
    print("Starting epoch {}:".format(epoch))
    dtcr_model.train(train_dl, test_dl)

Starting epoch 0:
[5] loss: 1.685743808746338, classify: 0.6931547045707702, recons: 0.9925890922546386
[10] loss: 1.6855220794677734, classify: 1.3863073110580444, recons: 1.984958577156067
Starting epoch 1:
[5] loss: 1.6853121519088745, classify: 0.6931510686874389, recons: 0.9921610593795777
[10] loss: 1.6853063344955443, classify: 1.3863011837005614, recons: 1.9843173027038574
Starting epoch 2:
